<!-- # -*- mode: markdown; coding: utf-8; fill-column: 60; org-indent-mode: t; column-number-mode: t; flyspell-mode: t; ispell-local-dictionary: "en"; eval: (visual-on); -*- -->

<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<link rel="stylesheet" href="style.css">


# EDAF75 -- lectures 1 and 2

In the text below there are two kinds of problems:

+ **Problem:**-marked problems, which I intend to solve during the lecture. Depending on how fast we progress, I may or may not have time to solve them all -- those of the problems we have to skip during the lectures are left as exercises (see below), but we can discuss them during QA sessions.

+ **Exercise:**-marked problems, which I suggest you solve yourselves (we can also work on them during the QA sessions).
  
-----

# Index

+ [Basic queries](#Basic-queries) (`SELECT-FROM-WHERE`, or SFW)
+ [Set operations](#Set-operations)
+ [Scalar functions and aggregate functions](#Scalar-functions-and-aggregate-functions)
+ [Grouping](#Grouping) (`GROUP BY`)
+ [Subqueries, Views and Common Table Expressions](#Subqueries,-Views-and-Common-Table-Expressions)
+ [Exercises to prepare for lecture 2](Exercises-to-prepare-for-lecture-2)
+ [Redundancy, and the case for splitting up tables](#Redundancy,-and-the-case-for-splitting-up-tables)
+ [Joining tables together](#Joining-tables-together)

-----


This document should be used as a [_Jupyter notebook_](https://jupyter.org/), it contains _cells_ in which we can evaluate program code (I assume most of you have used notebooks before, but I'll have a QA-session after the lecture, where you can ask if you have any questions about it).

Jupyter notebooks have built in support for _Julia_, _Python_, and _R_ (hence _Ju-Pyt-R_), here's some Python code:

In [1]:
def hello(name):
    print(f"hello, {name}!")

def main():
    name = input("What's your name: ")
    hello(name)
    
main()

What's your name:  Arvid


hello, Arvid!


You can run the code snippet above by clicking somewhere in the box, and press Shift-Enter.

We're primarily going to run SQL code (see below) in our notebooks, but I'll also show you some Python code later on in the course (you don't have to learn Python to take the course, though).


# Introduction to relational databases

If we were to keep track of all Nobel laureates in a Python or Java program, and didn't know about relational databases, we would probably define classes for the laureates, and put them in lists. We could also define classes for the categories, and have one list for each category, or have lists with one element per year, and somehow track all laureates in that year, or use some kind of dictionaries/maps. However we chose to keep track of the data, some searches, insertions and deletions would be easy to implement, and some would be cumbersome. We'd also have to be careful to keep our data consistent.

In this course, we'll use a technique which may at first seem too simple to be useful, but which turns out to be *incredibly* powerful. We're going to use [_relational databases_](https://en.wikipedia.org/wiki/Relational_database), and we'll store the data in 'simple' [_tables_](https://en.wikipedia.org/wiki/Table_(database)). Each table looks like a simple spreadsheet -- here is a table with some Nobel laureates:

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;border-color:#999;margin:0px auto}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:#999;color:#444;background-color:#F7FDFA;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:#999;color:#fff;background-color:#26ADE4;}
.tg .tg-e3zv{font-weight:bold}
.tg .tg-9hbo{font-weight:bold;vertical-align:top}
.tg .tg-yw4l{vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-e3zv">year</th>
    <th class="tg-9hbo">category</th>
    <th class="tg-9hbo">name</th>
    <th class="tg-9hbo">motivation</th>
  </tr>
  <tr>
    <td class="tg-yw4l">2011</td>
    <td class="tg-yw4l">Literature</td>
    <td class="tg-yw4l">Tomas Tranströmer</td>
    <td class="tg-yw4l">...</td>
  </tr>
  <tr>
    <td class="tg-yw4l">2011</td>
    <td class="tg-yw4l">Physics</td>
    <td class="tg-yw4l">Adam Riess</td>
    <td class="tg-yw4l">...</td>
  </tr>
  <tr>
    <td class="tg-yw4l">2011</td>
    <td class="tg-yw4l">Chemistry</td>
    <td class="tg-yw4l">Dan Shechtman</td>
    <td class="tg-yw4l">...</td>
  </tr>
  <tr>
    <td class="tg-yw4l">2011</td>
    <td class="tg-yw4l">Physiology or Medicine</td>
    <td class="tg-yw4l">Ralph Steinman</td>
    <td class="tg-yw4l">...</td>
  </tr>
</table>

A _row_ represents an item, and a _column_ represents a property of the items.

In the example above, each row describes a Nobel laurate, and for each laureate, we have columns showing what year the prize was awarded, in what category, the name of the laureate, and the motivation (not shown here).

One basic idea of relational databases is that all 'cells' in a table should be simple values (no lists or objects), and that we can use simple operations from [_relational algebra_](https://en.wikipedia.org/wiki/Relational_algebra) to get information from it.
We do it using a programming language which is highly specialized for manipulating and extracting information, it is called [SQL](https://en.wikipedia.org/wiki/SQL), which is short hand for _Structured Query Language_. SQL can be pronounced as either "S-Q-L", or "sequel".

SQL is divided into several sub-languages:

 + _Data Definition Language_ (_DDL_): constructs used to define the tables of a database,

 + _Data Manipulation Language_ (_DML_): statements used to query and manipulate data in a database,
   
 + _Transaction Control Language_ (_TCL_): commands used to handle transactions (we will return to what a transaction is later in the course), and
   
 + _Data Control Language_ (_DCL_): commands used to controll access to our data (we'll will not deal with them in this course).

This week we'll focus on "Data Manipulation", i.e., ways to query and modify our databases -- next week we'll look at how to design and create our databases, and then use DDL to define our tables.

We'll begin by discussing the following operations from relational algebra:

 + _selection_: choosing some of the rows of a table

 + _projection_: choosing some of the columns of a table

We will then see various ways to refine and combine queries.


## An actual DBMS

There are many different Relational Database Management Systems ([RDMBS:es](https://en.wikipedia.org/wiki/Relational_database)) which implements SQL, some of the most prominent are:

 *  [PostgreSQL](https://en.wikipedia.org/wiki/PostgreSQL)
 *  [MariaDB](https://en.wikipedia.org/wiki/MariaDB)
 *  [MySQL](https://en.wikipedia.org/wiki/MySQL)
 *  [Oracle](https://en.wikipedia.org/wiki/Oracle_Database)
 *  [Microsoft SQL Server](https://en.wikipedia.org/wiki/Microsoft_SQL_Server)
 *  [IBM DD2](https://en.wikipedia.org/wiki/IBM_Db2_Family)
 *  [SQLite](https://en.wikipedia.org/wiki/SQLite)

Most of the systems above are [_client-server_](https://en.wikipedia.org/wiki/Client%E2%80%93server_model)-systems, i.e., they have one program, a SQL server, which handles the data, and clients who communicate with the server in various ways. There are several different kinds of clients:

+ We can run an IDE, which allows us to see our tables in a GUI.

+ We can run command line clients (CLI) -- they are text based programs who work like typical REPLs, output will just be text in a terminal window.

+ We can write scripts which we send to the server, often through a CLI.
  
+ We can run a notebook (such as this one), and have it communicate with our database.

+ We can write code in a general purpose language, and have it communicate with our database.
  
In the course, we'll try all of these methods to access our databases.

The RDMBS we'll use in the course is [SQLite](https://en.wikipedia.org/wiki/SQLite), which is a lightweight but still very powerful system -- it is *by far* the most used RDBMS, and it's probably already running on all of your phones and computers (just as an example, if you use Chrome for browsing, your browsing history is typically saved in a SQL-database file `.config/google-chrome/Default/History `, and Mozilla use it for storing meta-data in Firefox and Thunderbird).
It's actually not a client/server system (instead it is a library which keeps our databases in files on our computer) -- but in the course, we'll think of SQLite as if it were a traditional client/server system, because in many ways, it behaves as one.

To be able to write SQL queries in this notebook, we first have to run:

In [8]:
pip install --user --upgrade ipython-sql

  Using cached ipython_sql-0.4.1-py3-none-any.whl (21 kB)
Processing /Users/arvidgramer/Library/Caches/pip/wheels/8c/76/0b/eb9eb3da7e2335e3577e3f96a0ae9f74f206e26457bd1a2bc8/prettytable-0.7.2-py3-none-any.whl
  Using cached sqlparse-0.4.3-py3-none-any.whl (42 kB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import sys
#sys.path.append("/Users/arvidgramer/.local/bin")
print(sys.path)

['/Users/arvidgramer/Desktop/F4/Databasteknik/lect01', '/Users/arvidgramer/opt/anaconda3/lib/python37.zip', '/Users/arvidgramer/opt/anaconda3/lib/python3.7', '/Users/arvidgramer/opt/anaconda3/lib/python3.7/lib-dynload', '', '/Users/arvidgramer/.local/lib/python3.7/site-packages', '/Users/arvidgramer/opt/anaconda3/lib/python3.7/site-packages', '/Users/arvidgramer/opt/anaconda3/lib/python3.7/site-packages/aeosa', '/Users/arvidgramer/opt/anaconda3/lib/python3.7/site-packages/IPython/extensions', '/Users/arvidgramer/.ipython']


In [1]:
%load_ext sql

The zip-archive in which this notebook is distributed has a file `lect01.sqlite` which contains all Nobel Laureates since 1901 -- to use it in our notebook, we import it with:

In [2]:
%sql sqlite:///lect01.sqlite

Now we're good to go, we just have to prefix our SQL queries with `%sql` (one line of SQL) or `%%sql` (several lines of SQL, this is the form we will use in most cases).


# Basic queries

A simple _SQL query_ can be written as:

~~~{.sql}
SELECT <what we're looking for>
FROM   <what table we're looking in>
~~~


Here `SELECT` is used to select all rows of a given table.

If we're only interesting in some of the rows, and we normally are, we write:

~~~{.sql}
SELECT <what we're looking for>
FROM   <what table we're looking in>
WHERE  <what items we're interested in>
~~~


The latter form is so common that it's got its own acronym: "SFW" (short for `SELECT`-`FROM`-`WHERE`).

You can see all versions of the `SELECT`-statement in SQLite on their [documentation for the `SELECT` statement](https://sqlite.org/lang_select.html) (there are corresponding pages for other commands).

If we want to see all columns in our rows, we can use

~~~{.text}
SELECT *
FROM   <what table we're looking in>
WHERE  <what items we're interested in>
~~~


This is sometimes considered 'sloppy', and we can use a projection (see above) to get just the columns we're interested in:

~~~{.text}
SELECT <column 1>, <column 2>, ...
FROM   <what table we're looking in>
WHERE  <what items we're interested in>
~~~


Observe that the selection (what rows we're interested in) is given in the `WHERE` clause, whereas the projection (what columns we're interested in) is defined in the `SELECT` clause (the naming is somewhat counter-intuitive).

Our Nobel Database contains the following information for each laureate:

 *  the _year_ the prize was awarded
 *  the _category_ ('chemistry', 'literature', 'physics', 'medicine')
 *  the _name_
 *  the _motivation_

Let's use the first form above to see all Nobel prizes which has been handed out:

In [17]:
%%sql
select *
FROM nobel
WHERE length(name) > 40

 * sqlite:///lect01.sqlite
Done.


year,category,name,motivation
1905,chemistry,Johann Friedrich Wilhelm Adolf von Baeyer,"in recognition of his services in the advancement of organic chemistry and the chemical industry, through his work on organic dyes and hydroaromatic compounds"
1911,literature,Count Maurice (Mooris) Polidore Marie Bernhard Maeterlinck,"in appreciation of his many-sided literary activities, and especially of his dramatic works, which are distinguished by a wealth of imagination and by a poetic fancy, which reveals, sometimes in the guise of a fairy tale, a deep inspiration, while in a mysterious way they appeal to the readers' own feelings and stimulate their imaginations"
1929,physics,Prince Louis-Victor Pierre Raymond de Broglie,for his discovery of the wave nature of electrons
1949,medicine,Antonio Caetano de Abreu Freire Egas Moniz,for his discovery of the therapeutic value of leucotomy in certain psychoses


This is too much to look through, so let's first limit the output to 10 rows (once again, look at the [documentation for `SELECT`](https://sqlite.org/lang_select.html), to see if you can find out how to do it).

If we limit the number of returned rows, it's often useful to start listing a number of rows down into the table (otherwise we could only see the first few rows when we used `LIMIT`) -- we use `OFFSET` to do that.

We can also select only those prizes awarded in 2013.

In [20]:
%%sql
SELECT *
FROM nobel
LIMIT 10
OFFSET 200

 * sqlite:///lect01.sqlite
Done.


year,category,name,motivation
1949,medicine,Walter Rudolf Hess,for his discovery of the functional organization of the interbrain as a coordinator of the activities of the internal organs
1949,physics,Hideki Yukawa,for his prediction of the existence of mesons on the basis of theoretical work on nuclear forces
1950,chemistry,Kurt Alder,for their discovery and development of the diene synthesis
1950,chemistry,Otto Paul Hermann Diels,for their discovery and development of the diene synthesis
1950,literature,Earl (Bertrand Arthur William) Russell,in recognition of his varied and significant writings in which he champions humanitarian ideals and freedom of thought
1950,medicine,Edward Calvin Kendall,"for their discoveries relating to the hormones of the adrenal cortex, their structure and biological effects"
1950,medicine,Philip Showalter Hench,"for their discoveries relating to the hormones of the adrenal cortex, their structure and biological effects"
1950,medicine,Tadeus Reichstein,"for their discoveries relating to the hormones of the adrenal cortex, their structure and biological effects"
1950,physics,Cecil Frank Powell,for his development of the photographic method of studying nuclear processes and his discoveries regarding mesons made with this method
1951,chemistry,Edwin Mattison McMillan,for their discoveries in the chemistry of the transuranium elements


Observe that the query returns a new table, we'll soon see that we can use the returned table in other queries.

**Problem:** _What year did Albert Einstein get his award, and why?_

In [22]:
%%sql
SELECT year, name as "Förnamn Efternamn", motivation
FROM nobel 
where name = "Albert Einstein"

 * sqlite:///lect01.sqlite
Done.


year,Förnamn Efternamn,motivation
1921,Albert Einstein,"for his services to Theoretical Physics, and especially for his discovery of the law of the photoelectric effect"


The names of the columns in the returned table is shown above the actual output, if we want to rename any of the columns in the returned table, we can use an _alias_:

In [ ]:
%%sql


**Problem:** _Who was awarded the physics prize in 1922?_

In [25]:
%%sql
SELECT year, name, category
FROM nobel 
WHERE year = 1922 and category = 'physics'

 * sqlite:///lect01.sqlite
Done.


year,name,category
1922,Niels Henrik David Bohr,physics


## Set operations

As was said above, the SQL language is built upon relational algebra, and sets are a first class citizen of relational algebra, so we can use set operations such as:

+ _union_ (`UNION`)
+ _intersection_ (`INTERSECT`)
+ _difference_ (`EXCEPT`)

We can use them to combine the results of two or more queries, _if the queries return tables of the same format_.

**Problem:** _Who were awarded the physics prize in 1922 and 1923?_ Try to solve this problem in at least three different ways, and see if you can do it using a set operation (however clumpsy it might be in this case).
You can look at the [documentation](https://sqlite.org/lang_select.html) to get some inspiration.

In [26]:
%%sql
SELECT name, category, year 
FROM nobel
WHERE category = 'physics' AND year in (1922, 1923)

 * sqlite:///lect01.sqlite
Done.


name,category,year
Niels Henrik David Bohr,physics,1922
Robert Andrews Millikan,physics,1923


In [28]:
%%sql
SELECT name, category, year 
FROM nobel
WHERE category = 'physics' AND year = 1922

UNION

SELECT name, category, year 
FROM nobel
WHERE category = 'physics' AND year = 1923

 * sqlite:///lect01.sqlite
Done.


name,category,year
Niels Henrik David Bohr,physics,1922
Robert Andrews Millikan,physics,1923


In [37]:
%%sql
SELECT name, category, year 
FROM nobel
WHERE year IN (1922, 1923)

EXCEPT

SELECT name, category, year
FROM nobel
WHERE category IN ('physics', 'medicine', 'literature')


 * sqlite:///lect01.sqlite
Done.


name,category,year
Francis William Aston,chemistry,1922
Fritz Pregl,chemistry,1923


In [53]:
%%sql
SELECT name, category, year 
FROM nobel
WHERE year IN (1922, 1923)

EXCEPT

SELECT DISTINCT category
FROM nobel

 * sqlite:///lect01.sqlite
(sqlite3.OperationalError) SELECTs to the left and right of EXCEPT do not have the same number of result columns
[SQL: SELECT name, category, year 
FROM nobel
WHERE year IN (1922, 1923)

EXCEPT

SELECT DISTINCT category
FROM nobel]
(Background on this error at: http://sqlalche.me/e/e3q8)


There are often several ways of doing things in SQL, and one of the main points of using SQL is that the database tries to optimize the operations it needs to fetch our data (there is some seriously clever code running behind the scenes).

**Problem:** _Who has been awarded the prize in literature since 2010, ordered by name?_

In [40]:
%%sql
SELECT name, category, year
FROM nobel
WHERE category = 'literature' AND year >= 2010
ORDER BY name

 * sqlite:///lect01.sqlite
Done.


name,category,year
Abdulrazak Gurnah,literature,2021
Alice Munro,literature,2013
Annie Ernaux,literature,2022
Bob Dylan,literature,2016
Kazuo Ishiguro,literature,2017
Louise Glück,literature,2020
Mario Vargas Llosa,literature,2010
Mo Yan,literature,2012
Olga Tokarczuk,literature,2018
Patrick Modiano,literature,2014


**Problem:** _What year did Winston Churchill win a prize, and in what category?_

In [46]:
%%sql
SELECT year, category, motivation
FROM nobel 
WHERE name LIKE '%Winston%'

 * sqlite:///lect01.sqlite
Done.


year,category,motivation
1953,literature,for his mastery of historical and biographical description as well as for brilliant oratory in defending exalted human values


Using `LIKE` in our conditions, we get some rudimentary form of wildcard matching (some SQL databases allow more advanced regular expressions, but that's beyond the scope of this course).

If we want to categorize our output, we can use a `CASE` statement, it has the general form:

~~~sql
SELECT ..., 
       CASE 
           WHEN ... THEN ...
           WHEN ... THEN ...
           ELSE ...
       END AS <name>
FROM ...
~~~


**Problem:** _Show all laureates in physics with a name beginning with 'P' -- if they won the prize before 1970 they're ancient, if the won the prize between 1970 and 2000 they're veterans, otherwise they're newbies._

In [50]:
%%sql
SELECT name, category,
    CASE
        WHEN year < 1970 THEN "ancient"
        WHEN year < 2000 THEN "veteran"
        WHEN year >= 2000 THEN "newbies"
    END AS "Street cred"
FROM nobel
WHERE name LIKE 'P%' AND category = 'physics'

 * sqlite:///lect01.sqlite
Done.


name,category,Street cred
Pieter Zeeman,physics,ancient
Pierre Curie,physics,ancient
Philipp Eduard Anton von Lenard,physics,ancient
Prince Louis-Victor Pierre Raymond de Broglie,physics,ancient
Paul Adrien Maurice Dirac,physics,ancient
Percy Williams Bridgman,physics,ancient
Patrick Maynard Stuart Blackett,physics,ancient
Polykarp Kusch,physics,ancient
Pavel Alekseyevich Cherenkov,physics,ancient
Philip Warren Anderson,physics,veteran


## `SELECT` and `SELECT DISTINCT`

**Problem:** _What are the different categories of Nobel prizes?_

In [55]:
%%sql
SELECT DISTINCT category
FROM nobel

 * sqlite:///lect01.sqlite
Done.


category
chemistry
literature
medicine
physics


Using `SELECT DISTINCT` we only get unique rows in our output table (duplicate rows are removed from the result).


## Scalar functions and aggregate functions

There are some functions we can apply to our values, each RDBMS supplies their own set of functions -- you can see some of SQLite's functions [here](https://sqlite.org/lang_corefunc.html).

**Problem:** _What was the initial letters of the laureates in year 2000?_ Hint: Use the [`substr`](https://sqlite.org/lang_corefunc.html#substr) function (and observe that the first character has index 1).

In [60]:
%%sql
select substr(name, 1, 1) as "First letter of name"
from nobel
where year = 2000

 * sqlite:///lect01.sqlite
Done.


First letter of name
A
A
H
G
A
E
P
H
J
Z


Here, the number of returned rows is the same as we would have had if we didn't apply the function.

An _aggregate function_ can be applied to all rows in a table, _but then returns only one value_.

The standard aggregate functions are:

 + `avg`: calculates the average for a given column
 + `count`: counts the rows in a given table
 + `min`: gets the minimum value of a given column
 + `max`: gets the maximum value of a given column
 + `sum`: calculates the sum of a given column

Observe that these are all functions which operates on several values, but return a single value. You can see a list of al SQLite's aggregate functions [here](https://sqlite.org/lang_aggfunc.html).

**Problem:** _How many laureates were there in year 2000?_

In [61]:
%%sql
select count(name)
from nobel 
where year = 2000

 * sqlite:///lect01.sqlite
Done.


count(name)
10


**Problem:** _How many of the laureates has had a first name beginning with an 'A'?_

In [63]:
%%sql
select count(name) as "Number of laureates with a name beginning with A"
from nobel
where name like "A%"

 * sqlite:///lect01.sqlite
Done.


Number of laureates with a name beginning with A
66


**Problem:** _What year was the first Nobel prize awarded?_

In [64]:
%%sql
select min(year)
from nobel

 * sqlite:///lect01.sqlite
Done.


min(year)
1901


**Exercise:** _How many Nobel prizes for chemistry have been awarded?_

In [65]:
%%sql
select count(name)
from nobel
where category = 'chemistry'

 * sqlite:///lect01.sqlite
Done.


count(name)
191


# Grouping

Using `GROUP BY` we can handle rows in groups -- to understand how it works, lets first look at the following query:

In [66]:
%%sql
SELECT    year, category, name
FROM      nobel
WHERE     year = 2013
ORDER BY  category

 * sqlite:///lect01.sqlite
Done.


year,category,name
2013,chemistry,Arieh Warshel
2013,chemistry,Martin Karplus
2013,chemistry,Michael Levitt
2013,literature,Alice Munro
2013,medicine,James E. Rothman
2013,medicine,Randy W. Schekman
2013,medicine,Thomas C. Südhof
2013,physics,François Englert
2013,physics,Peter W. Higgs


Here the rows of each category will end up adjacent to each other, and using `GROUP BY` we insert an invisible divider between the groups, and perform any aggregate function on the whole 'group':

In [70]:
%%sql
SELECT    category, count()
FROM      nobel
WHERE     year = 2013
GROUP BY  category

 * sqlite:///lect01.sqlite
Done.


category,count()
chemistry,3
literature,1
medicine,3
physics,2


So, if we apply an aggregate function, such as `count()`, in a table which we have grouped, _it will be applied to each group_, not to the whole table. Instead of getting one `count()` for the whole table (it would be a single value), we get one `count()` for each group (as above).

If we add `name` in the first line, we get a somewhat arbitrary result:

In [71]:
%%sql
SELECT    category, count(), name
FROM      nobel
WHERE     year = 2013
GROUP BY  category

 * sqlite:///lect01.sqlite
Done.


category,count(),name
chemistry,3,Arieh Warshel
literature,1,Alice Munro
medicine,3,James E. Rothman
physics,2,François Englert


The category and count is correct, but only one name is shown for each category.

The 'problem' is that we only get one row per group in the output, and that there may be several laureates in each group -- our query will return one of them in a seemingly haphazard manner. We can concatenate all names in the group using the [`group_concat`](https://sqlite.org/lang_aggfunc.html#groupconcat)-function:

In [72]:
%%sql
SELECT    category, count(), group_concat(name)
FROM      nobel
WHERE     year = 2013
GROUP BY  category

 * sqlite:///lect01.sqlite
Done.


category,count(),group_concat(name)
chemistry,3,"Arieh Warshel,Martin Karplus,Michael Levitt"
literature,1,Alice Munro
medicine,3,"James E. Rothman,Randy W. Schekman,Thomas C. Südhof"
physics,2,"François Englert,Peter W. Higgs"


There is no problem displaying `category` in the `SELECT`-statement above, we get a value which we know is the same for each row in the group (by definition, since that's what we grouped by).


If we're only interested in those categories with less than three laureates, we use `HAVING` to select only _groups_ with a given property:

In [73]:
%%sql
SELECT    category, count(), group_concat(name)
FROM      nobel
WHERE     year = 2013
GROUP BY  category
HAVING    count() < 3

 * sqlite:///lect01.sqlite
Done.


category,count(),group_concat(name)
literature,1,Alice Munro
physics,2,"François Englert,Peter W. Higgs"


This corresponds to a `WHERE` statement, but it applies to groups, not to individual rows (as `WHERE` does) -- so, _`WHERE` and `HAVING` have similar effects (they somehow narrow a search), but they're absolutely not interchangable!_

**Important** (and often misunderstood): In the query above we first have a `WHERE` statement to select some rows from the whole table, and then group the resulting selection.
_Every time we have both a `WHERE` and a `HAVING` in the same query, we must first use `WHERE` to select rows we can group, and then use `HAVING` to select groups._
We can use `WITH` statements or subqueries (see below) if we want to have it the other way around.


**Problem:** _How many laureates are there in each category?_

In [74]:
%%sql
select category, count() as "Number of laureates in each category"
from nobel
group by category

 * sqlite:///lect01.sqlite
Done.


category,Number of laureates in each category
chemistry,191
literature,119
medicine,225
physics,222


**Problem:** _Which categories have more than 200 laureates?_

In [79]:
%%sql
select category, count() as "Number of laureates in each category"
from nobel
group by category
having count() > 200

 * sqlite:///lect01.sqlite
Done.


category,Number of laureates in each category
medicine,225
physics,222


**Exercise:** _How many laureates were there each year between 1920 and 1930?_

In [81]:
%%sql
select year, count() 
from nobel
group by year
having year < 1930 and year > 1920

 * sqlite:///lect01.sqlite
Done.


year,count()
1921,3
1922,5
1923,5
1924,3
1925,4
1926,4
1927,5
1928,4
1929,6


**Exercise:** _Which years have seen more than 9 laureates?_

In [82]:
%%sql
select year, count() 
from nobel
group by year
having count() >9

 * sqlite:///lect01.sqlite
Done.


year,count()
1988,10
2000,10
2001,10
2002,10
2008,10
2009,10
2014,10
2017,10
2019,10


**Exercise:** _Which have been the 20 years with most laureates?_ (We don't need to be precise in case of ties.)

In [86]:
%%sql
select year, count() 
from nobel
group by year
order by -count()
limit 20

 * sqlite:///lect01.sqlite
Done.


year,count()
1988,10
2000,10
2001,10
2002,10
2008,10
2009,10
2014,10
2017,10
2019,10
1956,9


# Quick intro to window functions

As we saw above, we can treat partitions of our rows as 'groups', by using `GROUP BY`.

We also saw than when we use an aggregate function on a group, we collapse whole groups into one row in the output -- but sometimes we want to apply functions within a group of values, *and keep each row in the output*.

Above we listed all laureates in 2013, now we want to add one column to the output: for each laureate, we want to show how many laureates shared the prize in her or his category.

If we use `GROUP BY` and the aggregate function `count()` on the categories, we would only get one row per category, and using `count()` without grouping would collapse the whole result into just one row:

In [4]:
%%sql
SELECT    year, category, name, count() AS count      -- oh no!
FROM      nobel
WHERE     year = 2013
ORDER BY  category

 * sqlite:///lect01.sqlite
Done.


year,category,name,count
2013,chemistry,Arieh Warshel,9


Fortunately, SQL has introduced a way to apply functions only over 'partitions' of our tables, and keep all rows in the output -- using the reserved word `OVER` we can introduce a *window* of our rows, and apply the function only over this 'window':

In [5]:
%%sql
SELECT    year, category, name, count() OVER (PARTITION by category) AS count
FROM      nobel
WHERE     year = 2013
ORDER BY  category

 * sqlite:///lect01.sqlite
Done.


year,category,name,count
2013,chemistry,Arieh Warshel,3
2013,chemistry,Martin Karplus,3
2013,chemistry,Michael Levitt,3
2013,literature,Alice Munro,1
2013,medicine,James E. Rothman,3
2013,medicine,Randy W. Schekman,3
2013,medicine,Thomas C. Südhof,3
2013,physics,François Englert,2
2013,physics,Peter W. Higgs,2


We can also give our windows names, using an alias:

In [8]:
%%sql
SELECT    year, category, name, count() OVER categories AS count
FROM      nobel
WHERE     year = 2013
WINDOW    categories AS (PARTITION by category)
ORDER BY  -count

 * sqlite:///lect01.sqlite
Done.


year,category,name,count
2013,chemistry,Arieh Warshel,3
2013,chemistry,Martin Karplus,3
2013,chemistry,Michael Levitt,3
2013,medicine,James E. Rothman,3
2013,medicine,Randy W. Schekman,3
2013,medicine,Thomas C. Südhof,3
2013,physics,François Englert,2
2013,physics,Peter W. Higgs,2
2013,literature,Alice Munro,1


The aliased window definitions must come after any `WHERE` and `HAVING`, and before any `ORDER BY`.

So, if we use the reserved word `OVER` after a function, it will be applied according to a 'window' (there is more to it than this, but this will suffice for now). In the window we can:

+ define a *partition*, using `PARTITION BY`,
+ define an *order*, using `ORDER BY`, and
* define a *range*, which we can use to define groups of rows relative to each other (but we won't look at ranges in the course).

The function will be applied to each partition, in the same way we applied aggregate functions to groups above, but now we won't collapse the partitions. *Observe that the partitioning and ordering are based only on the selection we make (i.e., only those rows which are chosen in our `WHERE` clause).*

We can use our regular aggregate functions as window functions, but there are also a couple of dedicated window functions, such as (there are more, but we won't use them in the course):

+ `rank()`: ranks rows by the order specified in the window, ties can occur,
+ `row_number()`: as `rank()`, but now we avoid ties, and rank by row number in the output, and
+ `percent_rank()`: gives a value between 0.0 and 1.0 (so it's a bit of a misnomer), giving the row's relative rank within its partition.

You can find more [here](https://sqlite.org/windowfunctions.html).

Window functions can be very powerful, but we'll not delve too deeply into them in the course -- I want you to be aware of them, though!

**Problem:** Add one column which 'ranks' the laureates of 2013 in the table above according to the lengths of their names, within the categories -- shorter names should come before longer names.

In [21]:
%%sql
SELECT    year, category, name, rank() OVER (PARTITION BY category ORDER BY length(name)) AS count
FROM      nobel
WHERE     year = 2013
ORDER BY  category

 * sqlite:///lect01.sqlite
Done.


year,category,name,count
2013,chemistry,Arieh Warshel,1
2013,chemistry,Martin Karplus,2
2013,chemistry,Michael Levitt,2
2013,literature,Alice Munro,1
2013,medicine,James E. Rothman,1
2013,medicine,Thomas C. Südhof,1
2013,medicine,Randy W. Schekman,3
2013,physics,Peter W. Higgs,1
2013,physics,François Englert,2


**Problem:** For each laureate with the initial A, list their category, year, name, and 'freshness' within that category, i.e., the most recent laureate in a category is ranked 1, the second most recent laureate is ranked 2, etc. The ranks should be confined to laureates with the initial `A`.

In [64]:
%%sql
SELECT category, year, name, rank() OVER(PARTITION BY category ORDER BY -year) as freshness
FROM nobel
WHERE name LIKE "A%"

 * sqlite:///lect01.sqlite
Done.


category,year,name,freshness
chemistry,2019,Akira Yoshino,1
chemistry,2015,Aziz Sancar,2
chemistry,2013,Arieh Warshel,3
chemistry,2010,Akira Suzuki,4
chemistry,2009,Ada E. Yonath,5
chemistry,2004,Aaron Ciechanover,6
chemistry,2004,Avram Hershko,6
chemistry,2000,Alan G. MacDiarmid,8
chemistry,2000,Alan J. Heeger,8
chemistry,1999,Ahmed H. Zewail,10


# Some exercises

To spice things up a bit, I've included a table with all olympic games since 1896 -- the table `olympics` contains the columns:

+ `year`
+ `city`
+ `country`
+ `continent`
+ `season`
+ `ordinal_number`

If we look carefully at this table, we can find some unnecessary repetition, we will soon address this problem (but for now, we'll let it pass).

**Problem:** _How many olympic games have each continent hosted?_

In [30]:
%%sql
SELECT continent, count() as count
FROM olympics 
GROUP BY continent
ORDER BY -count

 * sqlite:///lect01.sqlite
Done.


continent,count
Europe,31
North America,12
Asia,8
Australia,2
South America,1


**Problem:** _When was the first olympic games in each continent?_

In [32]:
%%sql
select continent, min(year) as "first olympic"
from olympics
group by continent
order by "first olympic"

 * sqlite:///lect01.sqlite
Done.


continent,first olympic
Europe,1896
North America,1904
Australia,1956
Asia,1964
South America,2016


**Problem:** _For each olympic game, show how many olympic games had come before it in its continent_

In [45]:
%%sql
select year, city, continent, row_number() over (partition by continent order by year) as "number of previous games in continent"
from olympics 
order by year
limit 10

 * sqlite:///lect01.sqlite
Done.


year,city,continent,number of previous games in continent
1896,Athens,Europe,1
1900,Paris,Europe,2
1904,St. Louis,North America,1
1908,London,Europe,3
1912,Stockholm,Europe,4
1916,Berlin,Europe,5
1920,Antwerp,Europe,6
1924,Chamonix,Europe,7
1924,Paris,Europe,8
1928,St. Moritz,Europe,9


**Problem:** _Which countries have hosted the summer olympics more than once?_

In [49]:
%%sql
select country, count() as count
from olympics
group by country
having count > 1

 * sqlite:///lect01.sqlite
Done.


country,count
Australia,2
Austria,2
Canada,3
China,2
France,5
Germany,3
Greece,2
Italy,3
Japan,4
Norway,2


**Exercise:** _List the continents in descending order by the number of times they've hosted the summer olympics_

In [55]:
%%sql
select continent, count()
from olympics 
where season = "summer"
group by continent


 * sqlite:///lect01.sqlite
Done.


continent,count()
Asia,4
Australia,2
Europe,17
North America,6
South America,1


**Problem:** _Show a 'histogram' (no actual diagram, just the counts) over the the initial letter of the names of all Nobel laureates_

In [61]:
%%sql
select substr(name, 1, 1) as "bin", count() as count
from nobel
group by substr(name,1,1)

 * sqlite:///lect01.sqlite
Done.


bin,count
A,66
B,20
C,38
D,26
E,42
F,27
G,47
H,46
I,15
J,78


We can group by more than one column, by inserting invisible borders between all combinations of the given column values:

**Problem:** _Show a 'histogram' over the the initial letter of the names of all Nobel laureates, **for each category**_

In [78]:
%%sql
select category, substr(name, 1, 1), count()
from nobel
group by category, substr(name, 1, 1)


 * sqlite:///lect01.sqlite
Done.


category,"substr(name, 1, 1)",count()
chemistry,A,18
chemistry,B,3
chemistry,C,4
chemistry,D,6
chemistry,E,8
chemistry,F,10
chemistry,G,11
chemistry,H,12
chemistry,I,4
chemistry,J,20


**Problem:** _Has anyone won more than one Nobel prize?_ We can assume the names of the laureates are unique (so far they are!).

In [84]:
%%sql
select name, group_concat(category, ", "), count(), group_concat(year, ", ")
from nobel
group by name
having count()>1

 * sqlite:///lect01.sqlite
Done.


name,"group_concat(category, "", "")",count(),"group_concat(year, "", "")"
Frederick Sanger,"chemistry, chemistry",2,"1958, 1980"
John Bardeen,"physics, physics",2,"1956, 1972"
K. Barry Sharpless,"chemistry, chemistry",2,"2001, 2022"
"Marie Curie, née Sklodowska","physics, chemistry",2,"1903, 1911"


**Problem:** _Has anyone won more than one Nobel prize in the same category?_

In [86]:
%%sql
select name, group_concat(category, ", "), count(), group_concat(year, ", ")
from nobel
group by name, category
having count()>1

 * sqlite:///lect01.sqlite
Done.


name,"group_concat(category, "", "")",count(),"group_concat(year, "", "")"
Frederick Sanger,"chemistry, chemistry",2,"1958, 1980"
John Bardeen,"physics, physics",2,"1956, 1972"
K. Barry Sharpless,"chemistry, chemistry",2,"2001, 2022"


**Problem:** _Has anyone won Nobel prizes in different categories?_

In [89]:
%%sql
select name, group_concat(category, ", "), count(), group_concat(year, ", ")
from nobel
group by name
having count()>1

except

select name, group_concat(category, ", "), count(), group_concat(year, ", ")
from nobel
group by name, category
having count()>1

 * sqlite:///lect01.sqlite
Done.


name,"group_concat(category, "", "")",count(),"group_concat(year, "", "")"
"Marie Curie, née Sklodowska","physics, chemistry",2,"1903, 1911"


# Subqueries, Views and Common Table Expressions

As we noted above, the result of a `SELECT`-statement is itself a (kind of) table, and we can use such a table inside other statements.

One useful pattern is:

~~~sql
SELECT ...
FROM   ...
WHERE  ... IN
       (SELECT ...
        FROM ...
        WHERE ...)
~~~


The second query is called a _subquery_.

We'll use a subquery to find all literature laureates who split their prizes, we begin with a regular query:

**Problem:** _Which years were the Nobel prize for literature split?_

In [90]:
%%sql
SELECT    year
FROM      nobel
WHERE     category = 'literature'
GROUP BY  year
HAVING    count() > 1

 * sqlite:///lect01.sqlite
Done.


year
1904
1917
1966
1974


... and now we use the result of that query to find out what we're really looking for:

**Problem:** _Which literature laureates split their prizes?_

In [91]:
%%sql
SELECT    year, name
FROM      nobel
WHERE     category = 'literature'
          AND year IN (
              SELECT    year
              FROM      nobel
              WHERE     category = 'literature'
              GROUP BY  year
              HAVING    count() > 1)

 * sqlite:///lect01.sqlite
Done.


year,name
1904,Frédéric Mistral
1904,José Echegaray y Eizaguirre
1917,Henrik Pontoppidan
1917,Karl Adolph Gjellerup
1966,Nelly Sachs
1966,Shmuel Yosef Agnon
1974,Eyvind Johnson
1974,Harry Martinson


This can be simplified by using either of two ways to define 'temporary tables':

+ _Views_, and
+ _Common Table Expressions_ (a.k.a. a CTEs, or `WITH` statements).

They are the result of a `SELECT` statement, so for our problem above we could have created a view with the `CREATE VIEW` statement:

In [92]:
%%sql
CREATE VIEW shared_literature_prize(year) AS
  SELECT    year
  FROM      nobel
  WHERE     category = 'literature'
  GROUP BY  year
  HAVING    count() > 1

 * sqlite:///lect01.sqlite
Done.


[]

We can then use this view as if it was a regular table, in:

In [93]:
%%sql
SELECT    year, name
FROM      nobel
WHERE     category = 'literature'
          AND year IN (
              SELECT    year
              FROM      shared_literature_prize)

 * sqlite:///lect01.sqlite
Done.


year,name
1904,Frédéric Mistral
1904,José Echegaray y Eizaguirre
1917,Henrik Pontoppidan
1917,Karl Adolph Gjellerup
1966,Nelly Sachs
1966,Shmuel Yosef Agnon
1974,Eyvind Johnson
1974,Harry Martinson


Since `shared_literature_prize` has only one column, we can make the query even simpler (but beware that not all databases accept this simplification, and it only works in case we have exactly one column in our view):

In [94]:
%%sql
SELECT    year, name
FROM      nobel
WHERE     category = 'literature'
          AND year IN shared_literature_prize

 * sqlite:///lect01.sqlite
Done.


year,name
1904,Frédéric Mistral
1904,José Echegaray y Eizaguirre
1917,Henrik Pontoppidan
1917,Karl Adolph Gjellerup
1966,Nelly Sachs
1966,Shmuel Yosef Agnon
1974,Eyvind Johnson
1974,Harry Martinson


A view will stay around until we explicitly drop it:

In [95]:
%%sql
DROP VIEW shared_literature_prize

 * sqlite:///lect01.sqlite
Done.


[]

Common table expressions are in many ways similar to views, but they are sometimes more convenient:

+ they're defined as part of a `SELECT` statement (so there is nothing to drop afterwards),
+ since they're part of a `SELECT` statement, we only need one statement (which will become useful when we call our database remotely, we'll return to that later in the course), and
+ they can be defined recursively (we'll return to that later in the course).

Using a CTE, the solution to the problem above becomes:

In [106]:
%%sql
WITH shared_literature_prize(year) AS (
  SELECT    year
  FROM      nobel
  WHERE     category = 'literature'
  GROUP BY  year
  HAVING    count() > 1
)
SELECT year, name
from nobel
WHERE category = 'literature'
AND year IN (
              SELECT    year
              FROM      shared_literature_prize)

 * sqlite:///lect01.sqlite
Done.


year,name
1904,Frédéric Mistral
1904,José Echegaray y Eizaguirre
1917,Henrik Pontoppidan
1917,Karl Adolph Gjellerup
1966,Nelly Sachs
1966,Shmuel Yosef Agnon
1974,Eyvind Johnson
1974,Harry Martinson


or just (since our CTE has only one column):

In [102]:
%%sql
WITH shared_literature_prize(year) AS (
  SELECT    year
  FROM      nobel
  WHERE     category = 'literature'
  GROUP BY  year
  HAVING    count() > 1
)
select year, name
from nobel
WHERE     category = 'literature'
          AND year IN shared_literature_prize

 * sqlite:///lect01.sqlite
Done.


year,name
1904,Frédéric Mistral
1904,José Echegaray y Eizaguirre
1917,Henrik Pontoppidan
1917,Karl Adolph Gjellerup
1966,Nelly Sachs
1966,Shmuel Yosef Agnon
1974,Eyvind Johnson
1974,Harry Martinson


**Exercise:** _Show the years and categories for recurring laureates (i.e., laureates who has won more than once) -- use a CTE to do it._

In [107]:
%%sql
WITH recurrent_winners(name) AS (
  SELECT    name
  FROM      nobel
  GROUP BY  name
  HAVING    count() > 1
)

select name, year, category
from nobel 
where name in (
        select name 
        from recurrent_winners)

 * sqlite:///lect01.sqlite
Done.


name,year,category
"Marie Curie, née Sklodowska",1903,physics
"Marie Curie, née Sklodowska",1911,chemistry
John Bardeen,1956,physics
Frederick Sanger,1958,chemistry
John Bardeen,1972,physics
Frederick Sanger,1980,chemistry
K. Barry Sharpless,2001,chemistry
K. Barry Sharpless,2022,chemistry


**Exercise:** _Who has won the literature prize in a year when at least one chemistry laureate had a name beginning with 'L'?_ First try to solve this with a regular subquery, and then rewrite it using a CTE.

In [115]:
%%sql
select name, category
from nobel 
where category = 'literature' and year in (
    SELECT    year
    FROM      nobel
    WHERE     (category = 'chemistry') and (substr(name,1,1) like "L%")
)

 * sqlite:///lect01.sqlite
Done.


name,category
Frans Eemil Sillanpää,literature
Ernest Miller Hemingway,literature
Albert Camus,literature
Yasunari Kawabata,literature
Aleksandr Isayevich Solzhenitsyn,literature


In [120]:
%%sql
with chemistry_l_years(years) AS (
    select year
    from nobel
    where category = 'chemistry' and name like "L%"
)
select name, category 
from nobel
where year in chemistry_l_years and category = 'literature'

 * sqlite:///lect01.sqlite
Done.


name,category
Frans Eemil Sillanpää,literature
Ernest Miller Hemingway,literature
Albert Camus,literature
Yasunari Kawabata,literature
Aleksandr Isayevich Solzhenitsyn,literature


We saw above that we can't have another `WHERE` after the `HAVING` clause:

In [121]:
%%sql
SELECT    category, count() AS count
FROM      nobel
WHERE     year = 2013
GROUP BY  category
HAVING    count() < 3
WHERE     count > 1        --   <-- not allowed!

 * sqlite:///lect01.sqlite
(sqlite3.OperationalError) near "WHERE": syntax error
[SQL: SELECT    category, count() AS count
FROM      nobel
WHERE     year = 2013
GROUP BY  category
HAVING    count() < 3
WHERE     count > 1        --   <-- not allowed!]
(Background on this error at: http://sqlalche.me/e/e3q8)


but we can make our 'grouping query' into a subquery, and have another `WHERE` in the outer query

In [122]:
%%sql
SELECT category, count
FROM (
    SELECT    category, count() AS count
    FROM      nobel
    WHERE     year = 2013
    GROUP BY  category
    HAVING    count() < 3)
WHERE count > 1

 * sqlite:///lect01.sqlite
Done.


category,count
physics,2


A somewhat tidier way of expressing this is to use a `WITH`-statement:

In [123]:
%%sql
WITH category_count(category, count) AS (
    SELECT    category, count()
    FROM      nobel
    WHERE     year = 2013
    GROUP BY  category
    HAVING    count() < 3
)
SELECT category, count
FROM   category_count
WHERE  count > 1

 * sqlite:///lect01.sqlite
Done.


category,count
physics,2


## Correlated subqueries

Another form of subquery is:

~~~sql
SELECT ...,
       (SELECT ...
        FROM ...
        WHERE ...)
FROM   ...
~~~


This works if the subquery produces one result, such as when we use an aggregate function. As an example, solve the following problem:

**Problem:** _List the names of all laureates who has the longest name of all laureates in their category (in case of ties, all should be listed) -- order by category._

Here we can use a subquery which is 'run' for each row in
our main query:

In [124]:
%%sql
SELECT  category, year, name
FROM    nobel AS outer_nobel
WHERE   length(name) = (
            SELECT max(length(name))
            FROM   nobel
            WHERE  category = outer_nobel.category)
ORDER BY category

 * sqlite:///lect01.sqlite
Done.


category,year,name
chemistry,1905,Johann Friedrich Wilhelm Adolf von Baeyer
literature,1911,Count Maurice (Mooris) Polidore Marie Bernhard Maeterlinck
medicine,1949,Antonio Caetano de Abreu Freire Egas Moniz
physics,1929,Prince Louis-Victor Pierre Raymond de Broglie


This is called a _correlated subquery_ (since we refer to the enclosing query inside it). We use an alias to distinguish between the nobel table in the outer query and the nobel table in the subquery (it's the same table, but we 'iterate' through it separately).

BTW, we could have skipped the `AS` in

~~~sql
...
FROM    nobel AS outer_nobel
...
~~~


and just written:

~~~sql
...
FROM    nobel outer_nobel
...
~~~


The general opinion is that we should use `AS`, as it makes it more obvious that we're defining an alias.


<hr>

# Material to prepare for lecture 2

Some of the exercises in this section have answers right below them, try to solve the exercises yourself before looking at the answers.

**Exercise:** _Write a query to find out who has shared the chemistry prize with exactly one other laureate in years when the summer olympics were held in Europe?_

In [139]:
%%sql
with 
    two_chemistry(year) as (
        select year
        from nobel 
        where category = 'chemistry'
        group by year
        having count() = 2
    ),   
    summer_europe(year) as (
        select year
        from olympics
        where season = 'summer' and continent = 'Europe'
    )
select name, year, category
from nobel
where year in summer_europe and year in two_chemistry and category = 'chemistry'

 * sqlite:///lect01.sqlite
Done.


name,year,category
Paul Sabatier,1912,chemistry
Victor Grignard,1912,chemistry
Archer John Porter Martin,1952,chemistry
Richard Laurence Millington Synge,1952,chemistry
Brian K. Kobilka,2012,chemistry
Robert J. Lefkowitz,2012,chemistry


**Answer:** I'll solve this exercise during lecture 2.


**Exercise:** _Above we said that there is some unnecessary repetition in the olympics database, in what way?_

**Answer:** It is pretty obvious that countries never change continents (or at least it would literally take _eons_ of time to do so...), so repeating the fact that France is in Europe several times ...

In [140]:
%%sql
SELECT    country, continent, season, year, city
FROM      olympics
WHERE     country = 'France'
ORDER BY  year

 * sqlite:///lect01.sqlite
Done.


country,continent,season,year,city
France,Europe,summer,1900,Paris
France,Europe,summer,1924,Paris
France,Europe,winter,1924,Chamonix
France,Europe,winter,1968,Grenoble
France,Europe,winter,1992,Albertville


... is _redundant_.

And this redundance not only creates unnecessary repetition (and thus wastes disk space), it also introduces a risk of failures when we want to modify our table (we'll return to that later).

The 'problem' here is that we always get the same answer when we ask which continent a given country is in -- this is what we will call a _functional dependency_ when we get back to it in a few weeks time (when we talk about _normalization_).

The way we're going to solve the problem is to remove the continents from the `olympics`-table, and introduce another table, a kind of 'look-up'-table, to find the continent for each country -- we're going to do exactly that during this lecture.

There is another potential redundance in our `olympics` table -- every time the game has taken place in, say, Paris, it has been in France:

In [141]:
%%sql
SELECT    city, group_concat(year || ": " || country, ", ")
FROM      olympics
GROUP BY  city
HAVING    count() > 1
ORDER BY  city

 * sqlite:///lect01.sqlite
Done.


city,"group_concat(year || "": "" || country, "", "")"
Athens,"1896: Greece, 2004: Greece"
Beijing,"2022: China, 2008: China"
Berlin,"1916: Germany, 1936: Germany"
Innsbruck,"1964: Austria, 1976: Austria"
Lake Placid,"1932: United States, 1980: United States"
London,"1908: United Kingdom, 1948: United Kingdom, 2012: United Kingdom"
Los Angeles,"1932: United States, 1984: United States"
Paris,"1900: France, 1924: France"
St. Moritz,"1928: Switzerland, 1948: Switzerland"
Tokyo,"1964: Japan, 2020: Japan"


So, it does look as if we could extract the countries as well, and put them into a seperate 'look-up'-table, but if Sarajevo were to hold the olympics in the future, it wouldn't be in Yugoslavia anymore, and Moscow is no longer in the Soviet Union, so just extracting the countries as we did with continents above would not work. There is a kind of database called _temporal databases_, which is designed to handle cases such as this, but we'll only briefly touch upon them in this course.

And there is another, more fundamental reason for keeping both name and country: the names of cities aren't globally unique, there is a Paris in Texas, and a Moscow in Kansas, so the country helps us identify the cities uniquely (a _key_ is something which lets us identify values uniquely, we'll discuss keys next week, and then in week 4, when we give a more formal definition of the concept).


**Exercise:** _How could we add information about birth dates, and birth cities to our Nobel laureates?_

**Answer:** Since each laureate has one birth date, and one birth city, we can just add two more columns, `birth_date` and `birth_city` to our `nobel` table.


**Exercise:** _How could we add information about academic affiliations for the Nobel laureates?_ Some of the laureates have many affiliations, and some have none -- and observe that the 'cells' of our table must only contain simple values, so no lists or objects. Don't spend to much time to come up with a solution for this -- we haven't yet discussed the mechanism we're going to use to solve it -- but it's a good thing if you can see the limitations of what we've seen so far.

**Answer:** SQL allows us to use the value `NULL` when a value is missing, so for laureates without any affiliation, we can just use `NULL` (although doing it is somewhat controversial, and we can define our tables in a way which prohibits it -- more about that later).

But handling laureates with several affiliations is more difficult. One very primitive way of doing it, would be to add several columns: `affiliation_1`, `affiliation_2`, ..., but it's not only ugly, it only works if we limit the number of affiliations to a fixed number.

It turns out we can solve this problem in a manner similar to how we handled continents above: we can add another table (a 'look-up'-table), but this time the look-up table would have all affiliations for each laureate, so it could have several rows for each laureate.

<hr>

# Redundancy, and the case for splitting up tables

The term [redundancy](https://en.wikipedia.org/wiki/Redundancy_(linguistics)) can be defined in many contexts, in general it refers to information which is expressed more than once. It is sometimes desirable (the human body has an enormous amount of redundancy, that's what enable our bodies to self-heal), and many systems use redundancies for checks (the penultimate digit in your social security number is an example).

But for databases, redundancy is often a source of confusion and errors.
So, in this course, we'll generally try to avoid redundancies when we design our databases, and we're going to see two completely different way of doing it:

+ proper _ER-modeling_ (next week), and
+ _normalization_ (week 4).

Our original table of Olympic Games looks like this:

In [143]:
%%sql
SELECT *
FROM   olympics
limit 10

 * sqlite:///lect01.sqlite
Done.


year,city,country,continent,season,ordinal_number
1924,Chamonix,France,Europe,winter,I
1928,St. Moritz,Switzerland,Europe,winter,II
1932,Lake Placid,United States,North America,winter,III
1936,Garmisch-Partenkirchen,Germany,Europe,winter,IV
1948,St. Moritz,Switzerland,Europe,winter,V
1952,Oslo,Norway,Europe,winter,VI
1956,Cortina d'Ampezzo,Italy,Europe,winter,VII
1960,Squaw Valley,United States,North America,winter,VIII
1964,Innsbruck,Austria,Europe,winter,IX
1968,Grenoble,France,Europe,winter,X


We can use SQL to create a much better version of our olympic database with the following lines:

In [144]:
%%sql
DROP TABLE IF EXISTS continents;
CREATE TABLE continents (
  country   TEXT,
  continent TEXT NOT NULL,
  PRIMARY KEY (country)
);

INSERT OR IGNORE
INTO   continents(country, continent)
SELECT country, continent
FROM   olympics;

DROP TABLE IF EXISTS better_olympics;
CREATE TABLE  better_olympics (
  year            INT,
  city            TEXT,
  country         TEXT,
  season          TEXT,
  ordinal_number  TEXT,
  PRIMARY KEY (year, season)
);

INSERT
INTO   better_olympics(year, city, country, season, ordinal_number)
SELECT year, city, country, season, ordinal_number
FROM   olympics;

ALTER TABLE olympics
RENAME TO   redundant_olympics;
ALTER TABLE better_olympics
RENAME TO   olympics;

 * sqlite:///lect01.sqlite
Done.
Done.
26 rows affected.
Done.
Done.
54 rows affected.
Done.
Done.


1


Now we don't have to enter the continent the next time Athens, Paris or London is awarded the games.

Our two tables look like this:

In [145]:
%%sql
SELECT *
FROM   olympics;

 * sqlite:///lect01.sqlite
Done.


year,city,country,season,ordinal_number
1924,Chamonix,France,winter,I
1928,St. Moritz,Switzerland,winter,II
1932,Lake Placid,United States,winter,III
1936,Garmisch-Partenkirchen,Germany,winter,IV
1948,St. Moritz,Switzerland,winter,V
1952,Oslo,Norway,winter,VI
1956,Cortina d'Ampezzo,Italy,winter,VII
1960,Squaw Valley,United States,winter,VIII
1964,Innsbruck,Austria,winter,IX
1968,Grenoble,France,winter,X


In [146]:
%%sql
SELECT *
FROM   continents;

 * sqlite:///lect01.sqlite
Done.


country,continent
France,Europe
Switzerland,Europe
United States,North America
Germany,Europe
Switzerland,Europe
Norway,Europe
Italy,Europe
Austria,Europe
Japan,Asia
Yugoslavia,Europe


We've avoided some redundancy, but now we need a way to combine the information in our two tables, and that's what _joins_ are for.


# Joining tables together

To introduce joining, we're going to use a classic example: a database for handling college applications. In it a number of students applies for various majors at different colleges -- we want to keep track of:

 + student id (similar to stil-id)
 + student name
 + student's grade average (gpa)
 + the size of the student's highschool
 + the names of the colleges
 + the state for each college
 + the enrollment for each college
 + the major applied for
 + the decision ('Y' for accepted, 'N' otherwise)

One way of doing this would be to use one big table:

In [147]:
%%sql
SELECT *
FROM   big_college

 * sqlite:///lect01.sqlite
Done.


s_id,s_name,gpa,size_hs,c_name,state,enrollment,major,decision
123,Amy,3.9,1000,Berkeley,CA,36000,CS,Y
123,Amy,3.9,1000,Cornell,NY,21000,EE,Y
123,Amy,3.9,1000,Stanford,CA,15000,CS,Y
123,Amy,3.9,1000,Stanford,CA,15000,EE,N
234,Bob,3.6,1500,Berkeley,CA,36000,biology,N
345,Craig,3.5,500,Cornell,NY,21000,CS,Y
345,Craig,3.5,500,Cornell,NY,21000,EE,N
345,Craig,3.5,500,Cornell,NY,21000,bioengineering,N
345,Craig,3.5,500,MIT,MA,10000,bioengineering,Y
456,Doris,3.9,1000,None,None,None,None,None


But, just as with our olympics database above, this would give us a lot of redundance, so we'd like to split this big table into several related tables.

**Exercise:** In lectures 7 and 8 we'll discuss the theory and practice of splitting up tables -- for now, suggest tables which you think would make working with our data easier (don't look further down on this page until you've at least tried).

<p>&nbsp;</p>
<p>&nbsp;</p>
<p>&nbsp;</p>
<p>&nbsp;</p>
<p>&nbsp;</p>
<p>&nbsp;</p>

**Solution:** We can have one table for the students:

In [148]:
%%sql
SELECT *
FROM   students
LIMIT 4

 * sqlite:///lect01.sqlite
Done.


s_id,s_name,gpa,size_hs
123,Amy,3.9,1000
234,Bob,3.6,1500
345,Craig,3.5,500
456,Doris,3.9,1000


One for the colleges:

In [149]:
%%sql
SELECT *
FROM   colleges
LIMIT 4

 * sqlite:///lect01.sqlite
Done.


c_name,state,enrollment
Stanford,CA,15000
Berkeley,CA,36000
MIT,MA,10000
Cornell,NY,21000


And one for all collage applications made by the students:

In [150]:
%%sql
SELECT *
FROM   applications
LIMIT 4

 * sqlite:///lect01.sqlite
Done.


s_id,c_name,major,decision
123,Stanford,CS,Y
123,Stanford,EE,N
123,Berkeley,CS,Y
123,Cornell,EE,Y


The `applications` table uses `s_id` instead of `s_name`, since we could have several students with the same name (we actually have two 'Amy') -- `s_id` is guaranteed to be unique (we'll talk _much_ more about uniqueness next week).

Now assume we want to display all applications, _with the names of the students_ (not just the student id). We then need a way to combine information in the `applications` table with information in the `students` table, and we'll do it with a [_join_](https://en.wikipedia.org/wiki/Join_(SQL)).

The SQL standard specifies five different kinds of joins:

 + _cross join_
 + _inner join_
 + _left outer join_
 + _right outer join_
 + _full outer join_

## Cross join

The most primitive of the joins is the cross join -- we can write it as:

~~~
SELECT      *
FROM        a 
CROSS JOIN  b
~~~


Doing this, we get all combinations of rows from each of table `a` and `b` (a.k.a. the [_Cartesian product_](https://en.wikipedia.org/wiki/Cartesian_product)) -- the rows will contain all attributes from both tables (unless we make a projection).

If we do it with our `applications` and `students` tables we get:

In [155]:
%%sql
SELECT      *
FROM        applications 
CROSS JOIN  students
limit 10

 * sqlite:///lect01.sqlite
Done.


s_id,c_name,major,decision,s_id_1,s_name,gpa,size_hs
123,Stanford,CS,Y,123,Amy,3.9,1000
123,Stanford,CS,Y,234,Bob,3.6,1500
123,Stanford,CS,Y,345,Craig,3.5,500
123,Stanford,CS,Y,456,Doris,3.9,1000
123,Stanford,CS,Y,543,Craig,3.4,2000
123,Stanford,CS,Y,567,Edward,2.9,2000
123,Stanford,CS,Y,654,Amy,3.9,1000
123,Stanford,CS,Y,678,Fay,3.8,200
123,Stanford,CS,Y,765,Jay,2.9,1500
123,Stanford,CS,Y,789,Gary,3.4,800


The number of rows in this table is the product of the number of rows in each table, and most of the rows in the combined table are *totally* useless, since the `s_id`'s don't have anything to do with each other (e.g., in one line the fact that a student with `s_id` 123 has applied to CS at Stanford is combined with the fact that there is a student Bob with `s_id` 234 who has a `gpa` of 3.6).

The only rows of interest to us are those where the student id from the `applications` table is the same as the student id from the `students` table, and we can express that as:

In [156]:
%%sql
SELECT      *
FROM        applications 
CROSS JOIN  students
WHERE       applications.s_id = students.s_id

 * sqlite:///lect01.sqlite
Done.


s_id,c_name,major,decision,s_id_1,s_name,gpa,size_hs
123,Stanford,CS,Y,123,Amy,3.9,1000
123,Stanford,EE,N,123,Amy,3.9,1000
123,Berkeley,CS,Y,123,Amy,3.9,1000
123,Cornell,EE,Y,123,Amy,3.9,1000
234,Berkeley,biology,N,234,Bob,3.6,1500
345,MIT,bioengineering,Y,345,Craig,3.5,500
345,Cornell,bioengineering,N,345,Craig,3.5,500
345,Cornell,CS,Y,345,Craig,3.5,500
345,Cornell,EE,N,345,Craig,3.5,500
678,Stanford,history,Y,678,Fay,3.8,200


or, using aliases, so we don't have to write long table names:

In [157]:
%%sql
SELECT      *
FROM        applications AS a 
CROSS JOIN  students AS s
WHERE       a.s_id = s.s_id

 * sqlite:///lect01.sqlite
Done.


s_id,c_name,major,decision,s_id_1,s_name,gpa,size_hs
123,Stanford,CS,Y,123,Amy,3.9,1000
123,Stanford,EE,N,123,Amy,3.9,1000
123,Berkeley,CS,Y,123,Amy,3.9,1000
123,Cornell,EE,Y,123,Amy,3.9,1000
234,Berkeley,biology,N,234,Bob,3.6,1500
345,MIT,bioengineering,Y,345,Craig,3.5,500
345,Cornell,bioengineering,N,345,Craig,3.5,500
345,Cornell,CS,Y,345,Craig,3.5,500
345,Cornell,EE,N,345,Craig,3.5,500
678,Stanford,history,Y,678,Fay,3.8,200


We can clean up the output using a projection:

In [158]:
%%sql
SELECT      s.s_name, a.c_name, a.major
FROM        applications AS a 
CROSS JOIN  students AS s
WHERE       a.s_id = s.s_id

 * sqlite:///lect01.sqlite
Done.


s_name,c_name,major
Amy,Berkeley,CS
Amy,Cornell,EE
Amy,Stanford,CS
Amy,Stanford,EE
Bob,Berkeley,biology
Craig,Cornell,CS
Craig,Cornell,EE
Craig,Cornell,bioengineering
Craig,MIT,bioengineering
Craig,MIT,CS


or even

In [159]:
%%sql
SELECT      s_name AS name, c_name AS college, major
FROM        applications AS a
CROSS JOIN  students AS s
WHERE       a.s_id = s.s_id

 * sqlite:///lect01.sqlite
Done.


name,college,major
Amy,Berkeley,CS
Amy,Cornell,EE
Amy,Stanford,CS
Amy,Stanford,EE
Bob,Berkeley,biology
Craig,Cornell,CS
Craig,Cornell,EE
Craig,Cornell,bioengineering
Craig,MIT,bioengineering
Craig,MIT,CS


(we only need to prefix column names when there would otherwise be an ambiguity).

There is a shortcut for cross joins, we can just put a comma between the two tables we want to join:

In [160]:
%%sql
SELECT s_name AS name, c_name AS college, major
FROM   applications, students
WHERE  applications.s_id = students.s_id

 * sqlite:///lect01.sqlite
Done.


name,college,major
Amy,Berkeley,CS
Amy,Cornell,EE
Amy,Stanford,CS
Amy,Stanford,EE
Bob,Berkeley,biology
Craig,Cornell,CS
Craig,Cornell,EE
Craig,Cornell,bioengineering
Craig,MIT,bioengineering
Craig,MIT,CS


This works, but there are *much* more elegant ways to write this query.


## Inner joins

An _inner join_ combines two tables into a new table (it's not actually saved as a table) -- it does this by creating combined rows only when rows from the two tables 'match' each other. In this case it's the `s_id` columns which should match, so we write:

In [161]:
%%sql
SELECT      s_name AS name, c_name AS college, major
FROM        applications
INNER JOIN  students
ON          applications.s_id = students.s_id

 * sqlite:///lect01.sqlite
Done.


name,college,major
Amy,Berkeley,CS
Amy,Cornell,EE
Amy,Stanford,CS
Amy,Stanford,EE
Bob,Berkeley,biology
Craig,Cornell,CS
Craig,Cornell,EE
Craig,Cornell,bioengineering
Craig,MIT,bioengineering
Craig,MIT,CS


Logically it can be seen as the equivalent of a cross join and then a selection, but the database engine normally use algorithms which are much faster and requires much less memory when we define an inner join.
Once you get used to them, inner joins will feel much better to use than cross joins (although, in theory, the database should try to optimize all queries, and _might_ find an efficient way to execute even cross joins).

Inner joins are the default joins, so we can omit the word `INNER`, and just write `JOIN`:

In [162]:
%%sql
SELECT  s_name AS name, c_name AS college, major
FROM    applications
JOIN    students
ON      applications.s_id = students.s_id

 * sqlite:///lect01.sqlite
Done.


name,college,major
Amy,Berkeley,CS
Amy,Cornell,EE
Amy,Stanford,CS
Amy,Stanford,EE
Bob,Berkeley,biology
Craig,Cornell,CS
Craig,Cornell,EE
Craig,Cornell,bioengineering
Craig,MIT,bioengineering
Craig,MIT,CS


The condition on which we join tables is often called the _join predicate_, and a join where the join predicate is an equality test, such as here, is sometimes called an _equi-join_.
There is a special form of join when the columns we're comparing in an equi-join have the same name:

In [163]:
%%sql
SELECT  s_name AS name, c_name AS college, major
FROM    applications
JOIN    students
USING   (s_id)

 * sqlite:///lect01.sqlite
Done.


name,college,major
Amy,Berkeley,CS
Amy,Cornell,EE
Amy,Stanford,CS
Amy,Stanford,EE
Bob,Berkeley,biology
Craig,Cornell,CS
Craig,Cornell,EE
Craig,Cornell,bioengineering
Craig,MIT,bioengineering
Craig,MIT,CS


One benefit from using `USING` is that the columns we join over will not be duplicated, we get only one of them in the output.

I'd say this is the preferred way of writing this query.

There is an even simpler way to write it, using what's called a _natural join_, but it's error-prone, and you're *strongly* recommended not to use it.
It's written as:

In [164]:
%%sql
SELECT        s_name AS name, c_name AS college, major
FROM          applications
NATURAL JOIN  students

 * sqlite:///lect01.sqlite
Done.


name,college,major
Amy,Berkeley,CS
Amy,Cornell,EE
Amy,Stanford,CS
Amy,Stanford,EE
Bob,Berkeley,biology
Craig,Cornell,CS
Craig,Cornell,EE
Craig,Cornell,bioengineering
Craig,MIT,bioengineering
Craig,MIT,CS


and it joins the tables using an equi-join for all columns with coinciding names -- _this could create big problems if there are attributes in the tables which just happen to coincide without us realizing it!_
If we use the `JOIN` ... `USING` instead, we explicitly declare on which attributes to join, so we guard ourselves against accidental column name collisions.

Observe that we can use selections just as before when we join tables.

**Problem:** _Write a SQL query which shows all applications as above, but only for students applying for CS at Stanford._

In [170]:
%%sql
SELECT  s_name AS name, c_name AS college, major
FROM    applications
JOIN    students
on applications.s_id = students.s_id
where major ='CS' and college = 'Stanford'

 * sqlite:///lect01.sqlite
Done.


name,college,major
Amy,Stanford,CS
Irene,Stanford,CS
Helen,Stanford,CS


**Problem:** _Write a SQL query which shows the average gpa for students who have applied for each college._

This time it's OK to count applicants with several applications several times (so it becomes some kind of weighed average).

In [179]:
%%sql
select c_name as name, avg(gpa) as "Average GPA"
from applications
join students
using (s_id)
group by name

 * sqlite:///lect01.sqlite
Done.


name,Average GPA
Berkeley,3.733333333333333
Cornell,3.3666666666666667
MIT,3.6750000000000003
Stanford,3.683333333333333


Using a CTE (`WITH` statement), we can solve this problem in a more satisfactory way (not counting grades more than once):

In [17]:
%%sql
with 
    applicants(c_name, s_id) as (
        select distinct
        c_name, s_id
        from applications
    )

select c_name as name, avg(gpa) as "average gpa"
from applicants
join students
using (s_id)
group by name

 * sqlite:///lect01.sqlite
Done.


name,average gpa
Berkeley,3.733333333333333
Cornell,3.4333333333333336
MIT,3.6
Stanford,3.6399999999999997


We can join a table with itself, write a query which finds all pairs with the same Grade Point Average (GPA):

We can do it either with a cross join:

In [18]:
%%sql
SELECT s1.s_id, s1.s_name, s1.gpa, s2.s_id, s2.s_name, s2.gpa
FROM   students AS s1, students AS s2
WHERE  s1.gpa = s2.gpa AND
       s1.s_id < s2.s_id

 * sqlite:///lect01.sqlite
Done.


s_id,s_name,gpa,s_id_1,s_name_1,gpa_1
123,Amy,3.9,654,Amy,3.9
123,Amy,3.9,456,Doris,3.9
123,Amy,3.9,876,Irene,3.9
456,Doris,3.9,654,Amy,3.9
456,Doris,3.9,876,Irene,3.9
543,Craig,3.4,789,Gary,3.4
567,Edward,2.9,765,Jay,2.9
654,Amy,3.9,876,Irene,3.9


or with an inner join:

In [19]:
%%sql
SELECT s1.s_id, s1.s_name, s2.s_id, s2.s_name, gpa
FROM   students AS s1 
JOIN   students AS s2 
USING  (gpa)
WHERE  s1.s_id < s2.s_id

 * sqlite:///lect01.sqlite
Done.


s_id,s_name,s_id_1,s_name_1,gpa
123,Amy,654,Amy,3.9
123,Amy,456,Doris,3.9
123,Amy,876,Irene,3.9
456,Doris,654,Amy,3.9
456,Doris,876,Irene,3.9
543,Craig,789,Gary,3.4
567,Edward,765,Jay,2.9
654,Amy,876,Irene,3.9


Joining a table with itself is called a _self join_.
       
**Exercise:** Show the student names and majors for all applications to Stanford:

In [24]:
%%sql
select major, s_name
from applications
join students
using (s_id)
where c_name = 'Stanford'

 * sqlite:///lect01.sqlite
Done.


major,s_name
CS,Amy
EE,Amy
history,Fay
history,Jay
CS,Irene
CS,Helen


**Exercise:** Show the average `size_hs` for applications to the different colleges, order by descending size:

In [25]:
%%sql
select c_name, avg(size_hs)
from applications
join students
using (s_id)
group by c_name

 * sqlite:///lect01.sqlite
Done.


c_name,avg(size_hs)
Berkeley,1100.0
Cornell,916.6666666666666
MIT,825.0
Stanford,816.6666666666666


We can apply joins in several steps, each time combining what we've previously joined with another table (the result of which is a new table, which can be joined over and over again...).

**Exercise:** _Show the names of all students who have applied for a college in California -- also show the college and major._

In [28]:
%%sql
select s_name, c_name, major, state
from students
join applications
using (s_id)
join colleges
using (c_name)
where state = 'CA'

 * sqlite:///lect01.sqlite
Done.


s_name,c_name,major,state
Amy,Berkeley,CS,CA
Amy,Stanford,CS,CA
Amy,Stanford,EE,CA
Bob,Berkeley,biology,CA
Fay,Stanford,history,CA
Jay,Stanford,history,CA
Irene,Stanford,CS,CA
Helen,Berkeley,CS,CA
Helen,Stanford,CS,CA


In lab 1 you'll get plenty of exercise in joining.


### Using CTEs and inner joins to simplify queries

If we often use some query as part of other queries, we can save it using a _view_:

In [30]:
%%sql
DROP VIEW IF EXISTS application_info;
CREATE VIEW application_info(s_id, name, major, college, state) AS
  SELECT s_id, s_name, major, c_name, state
  FROM   applications
  JOIN   students
  USING  (s_id)
  JOIN   colleges
  USING  (c_name);

SELECT DISTINCT name, state
FROM   application_info
WHERE  s_id IN (123, 234, 456);

 * sqlite:///lect01.sqlite
Done.
Done.
Done.


name,state
Amy,CA
Amy,NY
Bob,CA


In lab 1 there are a couple of queries which would be easier to write if we had one big redundant view of all data -- try to define such a view (once you've come up with it, the following queries will be very simple one-liners).

Using joins we can also sometimes simplify unwieldly solutions with subqueries. We saw above that we could find all nobel laureates who had the longest name in each category (possibly tied), using the correlated subquery:

In [31]:
%%sql
SELECT  category, year, name
FROM    nobel AS outer_nobel
WHERE   length(name) = (
            SELECT max(length(name))
            FROM   nobel
            WHERE  category = outer_nobel.category)
ORDER BY category

 * sqlite:///lect01.sqlite
Done.


category,year,name
chemistry,1905,Johann Friedrich Wilhelm Adolf von Baeyer
literature,1911,Count Maurice (Mooris) Polidore Marie Bernhard Maeterlinck
medicine,1949,Antonio Caetano de Abreu Freire Egas Moniz
physics,1929,Prince Louis-Victor Pierre Raymond de Broglie


A nicer way of solving this is to first save the length of the longest name for each subquery in a temporary table, and then join this table using the category:

In [33]:
%%sql
WITH max_category_name_length(category, max_name_length) AS (
  SELECT   category, max(length(name))
  FROM     nobel
  GROUP BY category
)
SELECT year, category, name
FROM   nobel
JOIN   max_category_name_length
USING  (category)
WHERE  length(name) = max_name_length
ORDER BY  category, year, name

 * sqlite:///lect01.sqlite
Done.


year,category,name
1905,chemistry,Johann Friedrich Wilhelm Adolf von Baeyer
1911,literature,Count Maurice (Mooris) Polidore Marie Bernhard Maeterlinck
1949,medicine,Antonio Caetano de Abreu Freire Egas Moniz
1929,physics,Prince Louis-Victor Pierre Raymond de Broglie


In the joined table (`nobel JOIN max_category_name_length`), each row will get the additional `max_name_length` column, and it's the max length for the category of the award the row describes (since we joined using `category`).


## Outer joins

An inner join combines rows in different tables _when there is a match in the other table_, rows with no corresponding row in the other table will not turn up in the joined table.

With an _outer join_ we can make sure that every row in one or both of the tables turn up in the joined table -- in case there is no match, it will be combined with a row containing only `NULL` values (`NULL` is written as `None` in our notebooks).

As as example, we've seen how to join students and their applications:

In [34]:
%%sql
select s_name, c_name
from students
outer join applications

 * sqlite:///lect01.sqlite
(sqlite3.OperationalError) RIGHT and FULL OUTER JOINs are not currently supported
[SQL: select s_name, c_name
from students
outer join applications]
(Background on this error at: http://sqlalche.me/e/e3q8)


Now, assume we also want to see those students who haven't applied, we can do that using a _left outer join_:

In [35]:
%%sql
SELECT  s_id, s_name, c_name, major
FROM    students
    LEFT OUTER JOIN applications USING (s_id)

 * sqlite:///lect01.sqlite
Done.


s_id,s_name,c_name,major
123,Amy,Berkeley,CS
123,Amy,Cornell,EE
123,Amy,Stanford,CS
123,Amy,Stanford,EE
234,Bob,Berkeley,biology
345,Craig,Cornell,CS
345,Craig,Cornell,EE
345,Craig,Cornell,bioengineering
345,Craig,MIT,bioengineering
456,Doris,None,None


We still get the rows we got before, but now we also get rows with `NULL`'s in the attributes from the right relation, if there is no row in the right table which corresponds to a row in the left table (i.e., the `s_id` found in the `students` table has no match in the `applications` table -- the tuples on the left are sometimes called _dangling tuples_).

By the way, we can omit the `OUTER` keyword:

In [ ]:
%%sql


There is a 'natural' version of outer joins ...

In [ ]:
%%sql


... but for the same reasons as for inner joins, it's _much_, _much_ better to explicitly declare which attribute we're joining (using `USING`).

There is a way to get the same result without inner or outer joins, but it requires more code:

In [36]:
%%sql
SELECT      s_name, s.s_id, c_name, major
FROM        students s
CROSS JOIN  applications a
WHERE       s.s_id = a.s_id
UNION
SELECT      s_name, s_id, NULL, NULL
FROM        students
WHERE       s_id NOT IN (SELECT s_id
                         FROM   applications)

 * sqlite:///lect01.sqlite
Done.


s_name,s_id,c_name,major
Amy,123,Berkeley,CS
Amy,123,Cornell,EE
Amy,123,Stanford,CS
Amy,123,Stanford,EE
Amy,654,None,None
Bob,234,Berkeley,biology
Craig,345,Cornell,CS
Craig,345,Cornell,EE
Craig,345,Cornell,bioengineering
Craig,345,MIT,bioengineering


Seing this might help you understand what left outer join actually returns.

Now, the opposite problem, we want to see applications with no matching students -- of course we could just swap `students` and `applications` in the query above, but we could also use a _right outer join_:

In [37]:
%%sql
SELECT     s_name, s_id, c_name, major
FROM       students
RIGHT JOIN applications USING (s_id)

 * sqlite:///lect01.sqlite
(sqlite3.OperationalError) RIGHT and FULL OUTER JOINs are not currently supported
[SQL: SELECT     s_name, s_id, c_name, major
FROM       students
RIGHT JOIN applications USING (s_id)]
(Background on this error at: http://sqlalche.me/e/e3q8)


There is also a _full outer join_, which combines the left-
and the right outer joins:

In [ ]:
%%sql
SELECT    s_name, s_id, c_name, major
FROM      students
FULL JOIN applications USING (s_id)

Up until recently, SQLite only supported left outer joins, the other ones were left out :-), but since 2022 it supports left, right and full outer joins -- not having right outer join wasn't that much of an issue, since we can always turn a left outer join into a right outer join by taking the tables in reverse order.
If you run into problems using right or full outer joins in SQLite, you can either try to update SQLite (to at least 3.39.0), or just reverse the order of the tables (in case of a right outer join), or use `UNION` to combine two outer joins (in case of a full outer join).


**Exercise:** _Find all 'spurious' applications, i.e., applications where the student id doesn't match any student's id._ Try to solve this exercise in two ways (one using an outer join, the other using subqueries).

In [ ]:
%%sql


In [ ]:
%%sql


# Problems/Exercises

**Problem:** _How many olympic games have each continent hosted?_

In [43]:
%%sql
select *
from continents
limit 5

 * sqlite:///lect01.sqlite
Done.


country,continent
France,Europe
Switzerland,Europe
United States,North America
Germany,Europe
Switzerland,Europe


In [44]:
%%sql
select continent, count() as "Number of games hosted"
from olympics
join continents
using (country)
group by continent

 * sqlite:///lect01.sqlite
Done.


continent,Number of games hosted
Asia,8
Australia,2
Europe,31
North America,12
South America,1


**Problem:** _Some years no Nobel prize was awarded in some categories, list all such years/categories (we'll only deal with physics, chemistry, medicine and literature, since they're the only ones we have in our database...)._

Hint: To solve this problem we want to keep track of all possible prizes which could have been awarded, assuming all categories have been awarded since the outset (and indeed they have). So we want three things:

+ all years when prizes were awarded (in any category)
+ all categories (in our database...)
+ all combinations of years/categories

We can use a cross join to get all combinations:

In [53]:
%%sql
with 
    years_with_prizes(year) AS (
        SELECT distinct year
        FROM     nobel
    ),
    
    categories(category) as (
        select distinct category
        from nobel
    ),
    
    possible_years(year, category) as (
        select year, category
        from years_with_prizes
        cross join
        categories
    ),
    
    actual_years(year, category) as(
        select distinct year, category
        from nobel
    )
    
select category, year
from possible_years
where (year, category) not in actual_years
limit 10

 * sqlite:///lect01.sqlite
Done.


category,year
literature,1914
medicine,1915
chemistry,1916
medicine,1916
physics,1916
chemistry,1917
medicine,1917
literature,1918
medicine,1918
chemistry,1919


Once we have all possible combinations, we can use various techniques to find the missing ones:

In [ ]:
%%sql


Comment: Recently, SQLite has introduced a function [`generate_series`](https://sqlite.org/series.html), which lets us generate a series of values -- to generate a list of the years of 'potential' olympic summer games we can write (it was a recent addition, so it may not work in your notebook):

In [ ]:
%%sql
SELECT  value
FROM    generate_series(1900, 2000, 4)

During the world wars, no olympics were organized, so the list isn't quite right -- in the same manner we could have generated a list of all potential years for nobel prizes, but for some years during the wars, no prizes were awarded (not for the lack of worthy recipients).


**Problem:** _List the three programs (college/major) which has the highest gpa for the "last accepted" student._ (Assuming the students are accepted in order of decreasing gpa).

In [ ]:
%%sql


**Exercise:** _When was the first and the latest olympic games in each continent?_

In [ ]:
%%sql


**Exercise:** _Generate the unwieldly college application table we started out with above from the three smaller tables._

In [ ]:
%%sql


**Exercise:** _Recreate the original `olympics` table from the improved `olympics` and `continents`._

In [ ]:
%%sql


# Instead of lists

One of the 'rules' of relational databases is that the values in our tables must be primitive, we're not allowed to have lists or objects as values.
This requirement is called the ["First normal form"](https://en.wikipedia.org/wiki/First_normal_form), or "1NF" -- we'll return to 'normal forms' during lectures 7 and 8.

If we were to write a program to keep track of the phone numbers of our friends, we could write something like this in Java:

~~~java
class Friend {

    private String name;
    private List<PhoneNumber> phoneNumbers;
    
    ...
}

List<Friend> friends = ...
~~~


A literal translation of this into a table would be something like:

~~~
name      phone_numbers
----      -------------
Adam      [123456, 654321]
Bodil     [196811]
~~~


but this is in violation of the 'First normal form', so instead we have a new row for each phone number:

~~~
name      phone
----      -----
Adam      123456
Adam      654321
Bodil     196811
~~~


We can ask for Adam's number with:

~~~sql
SELECT phone
FROM   friends
WHERE  name = 'Adam'
~~~


and we'd get both numbers back:

~~~
phone
-----
123456
654321
~~~


Now, assume we also want to save the birthdays of our friends. We could add a column for birthdays like this:

~~~
name    birthday    phone
----    --------    -----
Adam    2 dec       123456
Adam    2 dec       654321
Bodil   30 nov      196811
~~~


What's the problem with this?

And what if we also want to save our friends' email addresses:

~~~
name        birthday    phone       email
----        --------    -----       -----
Adam        2 dec       123456      adam@gmail.com
Adam        2 dec       123456      adam@yahoo.com
Adam        2 dec       654321      adam@gmail.com
Adam        2 dec       654321      adam@yahoo.com
Bodil       30 nov      196811      bodil@itu.dk
Bodil       30 nov      196811      bodil@lu.se
Cecilia     9 apr       511235      cecilia@dn.se
Cecilia     9 apr       641587      cecilia@dn.se
David       12 jun      984531      david@gmail.co
David       12 jun      984531      david@hotmail
Emma        11 aug      123456      emma@lu.se
~~~


It may seem silly to have four rows for Adam, but if we removed one of them, we would loose vital information. If we removed the first row, and someone asked who had the phone number 123456 *and* the email address adam@gmail.com, the query:

~~~sql
SELECT  name, birthday
FROM    friends
WHERE   phone = '123456' AND email = 'adam@gmail.com'
~~~


would give an empty result.

The problem with the table above is that we have a lot of redundance, i.e., information is repeated in many places. This is not only a waste of storage space, it also makes it much, much harder to keep our tables up to date -- insertions, updates and removals affects many rows, and if we're not careful, we might leave the table in a corrupted state.

**Exercise:** _The data in the table above would be much easier to maintain if we split it into several tables -- suggest how to do that._